In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-03-16 02:50:59.874370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-1
    # Size x
    x_dim, y_dim = x.shape
    # Gradient vector
    if multioutput:
        gradient = np.zeros((x_dim, y_dim, y_dim))
    else:
        gradient = np.zeros((x_dim, y_dim, 1))
    for i in range(y_dim):
        # Vector for partial derivative estimation
        offset_tensor = np.zeros((x_dim, y_dim))
        offset_tensor[:, i] = h
        offset_tensor = tf.convert_to_tensor(offset_tensor,
                                             dtype = tf.float32)
        # Constantes:
        denominator = h
        if mode == 'progressive':
            numerator = model(
                tf.math.add(x, offset_tensor)
            ) - model(
                x
            ) 
        elif mode == 'regressive':
            numerator = model(
                x
            ) - model(
                tf.math.subtract(x, offset_tensor)
            )
        elif mode == 'centered':
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
        denominator = 2 * h
        gradient [:, i, :] = numerator / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [3]:
input_layer = keras.Input(shape = (2,), name='input_nn')
output_layer = layers.Dense(
    units = 1, 
    activation = 'relu', 
    name = 'first_dense'
)(input_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

2023-03-16 02:51:21.888769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# Domain -5:5
x = tf.convert_to_tensor(
    np.array(range(0,20)).reshape((10,2)),
    dtype = tf.float32    
)
# Test 
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(xs)
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})

In [18]:
custom_grads = custom_derivative_model(x, custom_model, multioutput=False)

In [19]:
custom_grads

<tf.Tensor: shape=(10, 2, 1), dtype=float32, numpy=
array([[[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.1179982 ],
        [ 0.        ]],

       [[ 0.9054357 ],
        [-0.06694436]]], dtype=float32)>

In [20]:
grads

{'x': <tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.       , -0.       ],
        [ 0.9054364, -0.066944 ]], dtype=float32)>}